In [1]:
import cv2
import numpy as np
import os
import face_recognition
import ctypes

In [2]:
#Function to lock the screen
def lock_screen():
    if os.name == 'nt':  # Windows
        ctypes.windll.user32.LockWorkStation()
    elif os.name == 'posix':  # Mac/Linux
        os.system('gnome-screensaver-command -l')

In [4]:
# Load known faces
known_faces_dir = r"Pics"
known_face_encodings = []
known_face_names = []

for filename in os.listdir(known_faces_dir):
    img_path = os.path.join(known_faces_dir, filename)
    img = face_recognition.load_image_file(img_path)
    
    # Check if face encodings are found
    face_encodings = face_recognition.face_encodings(img)
    if face_encodings:
        img_encoding = face_encodings[0]
        known_face_encodings.append(img_encoding)
        known_face_names.append(filename)
    else:
        print(f"No faces found in {filename}")

In [5]:
# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    stranger_detected = False
    tolerance = 0.6  

    for face_encoding in face_encodings:
        # Compute distances between the current face encoding and known face encodings
        distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        
        # Check if any of the distances are within the tolerance
        matches = distances <= tolerance
        
        if not any(matches):
            stranger_detected = True
            break

    if stranger_detected:
        print("Stranger detected! Locking screen.")
        lock_screen()
        break
    # Draw rectangles around detected faces
    for (top, right, bottom, left) in face_locations:
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
    cv2.imshow('Face Recognition Security', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Stranger detected! Locking screen.
